Imports

In [1]:
from pulp import LpProblem, LpVariable, LpBinary, lpSum, LpMaximize, LpStatus, PULP_CBC_CMD
from datetime import datetime, timedelta
import pandas as pd

Data Preprocessing

In [2]:
# extract data
courses = pd.read_csv('../courses.csv')
venues = pd.read_csv('../venues.csv')

# drop unavailable venues
available_venues = venues[venues['isAvailable'] == 'yes']

# convert time to datetime and sort
courses['Start Time'] = pd.to_datetime(courses['Start Time'], format='%I:%M %p')
courses['End Time'] = pd.to_datetime(courses['End Time'], format='%I:%M %p')
courses = courses.sort_values(by='Start Time')

# separate MW and TR courses
MW_courses = courses[courses['Days'] == 'MW']
TR_courses = courses[courses['Days'] == 'TR']

# reset index
MW_courses.reset_index(drop=True, inplace=True)
TR_courses.reset_index(drop=True, inplace=True)

Helper Functions

In [3]:
def weighted_sum(courses, i, venues, j):
    w1 = courses['Enrollment'][i] / venues['Seating Capacity'][j]   # space utilization ratio
    return w1   # + w2 + w3 + ...

# more weights can be added to fine-tune the optimization
# e.g. instructor's preference, specific venue requirement (labs, exams etc.),
# availability of venue based on time of day (8am to 5pm can be divided into 3 weighted segments).

In [4]:
def cook_schedule(courses, venues):
    """Returns the optimal schedule for the given courses and venues."""

    # Create the problem
    problem = LpProblem("Class-Scheduling", LpMaximize)

    # DVs: each course can be scheduled in one of the available venues
    x = LpVariable.dicts("x", [(i, j) for i in range(len(courses)) for j in range(len(available_venues))], 0, 1, LpBinary)

    # Objective Function: Maximize the average space utilization for all venues (current goal)
    problem += lpSum(x[i, j] * weighted_sum(courses, i, venues, j) for i in range(len(courses)) for j in range(len(available_venues)))

    # Constraint: Each course is scheduled in exactly one venue
    for i in range(len(courses)):
        problem += lpSum(x[i, j] for j in range(len(available_venues))) == 1

    # Constraint: Assigned venue has enough seating capacity
    for i in range(len(courses)):
        for j in range(len(available_venues)):
            if courses['Enrollment'][i] > available_venues['Seating Capacity'][j]:
                problem += x[i, j] == 0

    # Constraint: No overlapping courses in the same venue
    overlapping_slots = []
    for i in range(len(courses)):
        overlaps = [k for k in range(i, len(courses)) if courses['Start Time'][k] < courses['End Time'][i] and courses['End Time'][k] > courses['Start Time'][i]]
        if len(overlaps) > 1 and True not in [set(overlaps).issubset(slot) for slot in overlapping_slots]:
            overlapping_slots.append(overlaps)

    for j in range(len(available_venues)):
        for overlap in overlapping_slots:
            problem += lpSum(x[i, j] for i in overlap) <= 1

    # Solve the problem
    problem.solve(PULP_CBC_CMD())

    # Optimal schedule
    return x

In [ ]:
def save_results(schedule, stats, courses, venues):
    """Save the optimal schedule to a file."""

    x = cook_schedule(courses, venues)  # get the optimal schedule
    
    # convert time to string
    courses['Start Time'] = courses['Start Time'].dt.strftime('%I:%M %p')
    courses['End Time'] = courses['End Time'].dt.strftime('%I:%M %p')

    # write schedule to file
    with open(f'{schedule}', 'w') as file:
        file.write("Course Code,Start Time,End Time,Venue,Enrollment,Seating Capacity\n")
        for i in range(len(courses)):
            for j in range(len(venues)):
                if x[i, j].value() == 1:
                    file.write(f"{courses['Course Code'][i]},{courses['Start Time'][i]},{courses['End Time'][i]},{venues['Class Venues'][j]},{courses['Enrollment'][i]},{venues['Seating Capacity'][j]}\n")

    # venue stats for visualization
    with open(f'{stats}', 'w') as file:
        file.write("Venue,Seating Capacity,Classes Scheduled,Average Space Utilization %\n")
        for j in range(len(venues)):
            classes_scheduled = sum(x[i, j].value() for i in range(len(courses)))
            file.write(f"{venues['Class Venues'][j]}, {venues['Seating Capacity'][j]}, {classes_scheduled}, {sum(x[i, j].value() * (courses['Enrollment'][i] * 100/ venues['Seating Capacity'][j]) for i in range(len(courses)))/sum((x[i, j].value() for i in range(len(courses)))):.2f}\n")

In [ ]:
# moment of truth
save_results('schedule_MW.csv', 'stats_MW.csv', MW_courses, available_venues)
save_results('schedule_TR.csv', 'stats_TR.csv', TR_courses, available_venues)